# Get the Exact Answer
Start off by computing the exact Hessian to use a reference point. 
First relax the structure then compute the Hessians using [ase's Vibrations module](https://databases.fysik.dtu.dk/ase/ase/vibrations/modes.html), which will compute them numerically using central derivatives

In [1]:
from ase.thermochemistry import IdealGasThermo
from ase.vibrations import VibrationsData, Vibrations
from ase.calculators.mopac import MOPAC
from ase.calculators.psi4 import Psi4
from ase.optimize import QuasiNewton
from ase import Atoms, units
from ase.io import write
from contextlib import redirect_stderr
from time import perf_counter
from platform import node
from pathlib import Path
from os import devnull
import numpy as np
import shutil
import json
import os

Configuration

In [2]:
molecule_name = 'water'
method = 'pm7'
mopac_methods = ['pm7']  # Use MOPAC for these methods
basis = None  # Set to None for MOPAC methods
threads = min(os.cpu_count(), 12)
assert (method in mopac_methods) == (basis is None), 'Use a basis of None for MOPAC computations'

Derived

In [3]:
run_name = f'{molecule_name}_{method}_{basis}'

## Load in Target Molecule
We have it in a JSON file from PubChem

In [4]:
def load_molecule(name: str) -> Atoms:
    """Load a molecule from a PubChem JSON file
    
    Args:
        name: Name of the molecule
    Returns:
        ASE Atoms object
    """
    
    # Get the compound data
    with open(f'data/structures/{name}.json') as fp:
        data = json.load(fp)
    data = data['PC_Compounds'][0]
        
    # Extract data from the JSON
    atomic_numbers = data['atoms']['element']
    positions = np.zeros((len(atomic_numbers), 3))
    conf_data = data['coords'][0]['conformers'][0]
    for i, c in enumerate('xyz'):
        if c in conf_data:
            positions[:, i] = conf_data[c]
        
    # Build the object    
    return Atoms(numbers=atomic_numbers, positions=positions)

In [5]:
atoms = load_molecule(molecule_name)

## Perform the Geometry Optimization
Build the ASE calculator then run QuasiNewton to a high tolerance

In [6]:
if method not in mopac_methods:
    calc = Psi4(method=method, basis=basis, num_threads=threads, memory='4096MB')
else:
    calc = MOPAC(method=method, command='mopac PREFIX.mop > /dev/null')

In [7]:
%%time
atoms.calc = calc
dyn = QuasiNewton(atoms)
with redirect_stderr(devnull):
    dyn.run(fmax=0.01)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 13:34:20        3.054451*      39.6081
BFGSLineSearch:    1[  1] 13:34:20       -2.442459*       1.4363
BFGSLineSearch:    2[  3] 13:34:20       -2.504294*       0.3803
BFGSLineSearch:    3[  5] 13:34:20       -2.506296*       0.0708
BFGSLineSearch:    4[  6] 13:34:20       -2.506439*       0.0029
CPU times: user 35.5 ms, sys: 20.8 ms, total: 56.3 ms
Wall time: 342 ms


Save the output file

In [8]:
out_dir = Path('data') / 'exact'
out_dir.mkdir(exist_ok=True)

In [9]:
write(out_dir / f'{run_name}.xyz', atoms)

## Compute the Hessian using ASE
ASE has a built-in method which uses finite displacements

In [10]:
if Path('vib').is_dir():
    shutil.rmtree('vib')

In [11]:
%%time
finite_diff_time = perf_counter()
vib = Vibrations(atoms)
vib.run()
finite_diff_time = perf_counter() - finite_diff_time

CPU times: user 26.8 ms, sys: 42.5 ms, total: 69.4 ms
Wall time: 782 ms


Save the vibration data

In [12]:
vib_data = vib.get_vibrations()
with (out_dir / f'{run_name}-ase.json').open('w') as fp:
    vib_data.write(fp)

Print the ZPE for reference

In [13]:
vib_data.get_zero_point_energy()

0.4429860245936006

## Repeat with Psi4's analytic derivatives
See if we get the same answer faster

In [14]:
if isinstance(calc, Psi4):
    # Compute
    analytic_time = perf_counter()
    calc.set_psi4(atoms)
    hess = calc.psi4.hessian(f'{method}/{basis}')
    analytic_time = perf_counter() - analytic_time

    # Convert to ASE format
    analytic_hess = hess.to_array() * units.Hartree / units.Bohr / units.Bohr
    vib_data = VibrationsData.from_2d(atoms, analytic_hess)
    with (out_dir / f'{run_name}-psi4.json').open('w') as fp:
        vib_data.write(fp)
else:
    analytic_time = None

Save the runtimes

In [15]:
with (out_dir / f'{run_name}-times.json').open('w') as fp:
    json.dump({
        'hostname': node(),
        'finite-diff': finite_diff_time,
        'analytic': analytic_time,
    }, fp)